# 4.2 Classification Linéaire en Pratique

## PSY3913/6913 - IA, Psychologie et Neuroscience Cognitive

Dans ce notebook, nous allons explorer la classification linéaire (régression logistique) :
1. Comprendre la fonction sigmoïde
2. Implémenter la régression logistique from scratch
3. Visualiser les frontières de décision
4. Utiliser scikit-learn
5. Classification multi-classe
6. Application en neurosciences

## 1. Import des bibliothèques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import seaborn as sns

# Pour des graphiques plus jolis
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Pour la reproductibilité
np.random.seed(42)

## 2. Comprendre la fonction sigmoïde

La fonction sigmoïde est au cœur de la régression logistique. Elle transforme n'importe quelle valeur réelle en une valeur entre 0 et 1.

$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

In [ ]:
def sigmoid(z):
    """
    Fonction sigmoïde.
    
    Parameters:
    -----------
    z : float or array-like
        Valeur(s) d'entrée
    
    Returns:
    --------
    float or array-like
        Valeur(s) entre 0 et 1
    """
    return 1 / (1 + np.exp(-z))

# Visualisation de la fonction sigmoïde
z_values = np.linspace(-10, 10, 200)
sigmoid_values = sigmoid(z_values)

plt.figure(figsize=(10, 6))
plt.plot(z_values, sigmoid_values, linewidth=2, label='σ(z)')
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='Seuil de décision (0.5)')
plt.axvline(x=0, color='g', linestyle='--', alpha=0.5)
plt.xlabel('z = Wx + b', fontsize=12)
plt.ylabel('σ(z)', fontsize=12)
plt.title('Fonction Sigmoïde', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.ylim([-0.1, 1.1])

# Annoter des points clés
plt.annotate('σ(0) = 0.5', xy=(0, 0.5), xytext=(2, 0.3),
            arrowprops=dict(arrowstyle='->', color='black'),
            fontsize=11)
plt.annotate('σ(∞) → 1', xy=(8, sigmoid(8)), xytext=(5, 0.85),
            arrowprops=dict(arrowstyle='->', color='black'),
            fontsize=11)
plt.annotate('σ(-∞) → 0', xy=(-8, sigmoid(-8)), xytext=(-5, 0.15),
            arrowprops=dict(arrowstyle='->', color='black'),
            fontsize=11)

plt.show()

print("💡 Propriétés de la sigmoïde:")
print("   • Sortie toujours entre 0 et 1 → peut être interprétée comme probabilité")
print("   • σ(0) = 0.5 → point de décision")
print("   • Forme en 'S' → transition douce entre les classes")
print(f"   • σ(5) = {sigmoid(5):.4f} → haute confiance pour classe 1")
print(f"   • σ(-5) = {sigmoid(-5):.4f} → haute confiance pour classe 0")

## 3. Créer des données synthétiques 2D

Créons un dataset simple avec 2 features et 2 classes linéairement séparables.

In [ ]:
# Générer des données synthétiques
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0, 
                           n_informative=2, n_clusters_per_class=1,
                           class_sep=1.5, random_state=42)

print(f"📊 Dataset créé:")
print(f"   Forme de X: {X.shape}")
print(f"   Forme de y: {y.shape}")
print(f"   Classes: {np.unique(y)}")
print(f"   Distribution: Classe 0: {np.sum(y==0)}, Classe 1: {np.sum(y==1)}")

# Visualisation
plt.figure(figsize=(10, 6))
plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Classe 0', alpha=0.6, edgecolors='k')
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Classe 1', alpha=0.6, edgecolors='k')
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('Dataset pour classification binaire', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 Observation: Les deux classes sont relativement bien séparables.")
print("   Une ligne devrait pouvoir les séparer efficacement.")

## 4. Implémentation from scratch : Régression logistique

Implémentons la régression logistique avec la descente de gradient.

**Rappel :**
- Modèle : $\hat{y} = \sigma(Wx + b)$
- Loss : $L = -\frac{1}{n} \sum [y \log(\hat{y}) + (1-y) \log(1-\hat{y})]$
- Gradients : 
  - $\frac{\partial L}{\partial w} = \frac{1}{n} X^T(\hat{y} - y)$
  - $\frac{\partial L}{\partial b} = \frac{1}{n} \sum (\hat{y} - y)$

In [ ]:
class LogisticRegressionGD:
    """
    Régression logistique avec descente de gradient.
    """
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.lr = learning_rate
        self.n_iterations = n_iterations
        self.w = None
        self.b = None
        self.loss_history = []
    
    def sigmoid(self, z):
        """Fonction sigmoïde."""
        return 1 / (1 + np.exp(-z))
    
    def compute_loss(self, y, y_pred):
        """
        Calculer la binary cross-entropy loss.
        
        Ajoute un petit epsilon pour éviter log(0).
        """
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
        return loss
    
    def fit(self, X, y):
        """
        Entraîner le modèle.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
        y : array-like, shape (n_samples,)
        """
        n_samples, n_features = X.shape
        
        # Initialisation
        self.w = np.zeros((n_features, 1))
        self.b = 0
        
        # Reshape y
        y = y.reshape(-1, 1)
        
        # Descente de gradient
        for i in range(self.n_iterations):
            # Prédiction
            z = X.dot(self.w) + self.b
            y_pred = self.sigmoid(z)
            
            # Calcul de la loss
            loss = self.compute_loss(y, y_pred)
            self.loss_history.append(loss)
            
            # Calcul des gradients
            dw = (1 / n_samples) * X.T.dot(y_pred - y)
            db = (1 / n_samples) * np.sum(y_pred - y)
            
            # Mise à jour des paramètres
            self.w -= self.lr * dw
            self.b -= self.lr * db
            
            # Affichage occasionnel
            if (i + 1) % 100 == 0:
                print(f"Itération {i+1}/{self.n_iterations}, Loss: {loss:.4f}")
    
    def predict_proba(self, X):
        """
        Prédire les probabilités.
        """
        z = X.dot(self.w) + self.b
        return self.sigmoid(z)
    
    def predict(self, X, threshold=0.5):
        """
        Prédire les classes.
        """
        y_pred_proba = self.predict_proba(X)
        return (y_pred_proba >= threshold).astype(int)
    
    def get_params(self):
        """
        Retourner les paramètres.
        """
        return {'w': self.w, 'b': self.b}

### Entraînement du modèle

In [ ]:
# Créer et entraîner le modèle
model_scratch = LogisticRegressionGD(learning_rate=0.1, n_iterations=1000)
print("Entraînement du modèle...\n")
model_scratch.fit(X, y)

# Récupérer les paramètres
params = model_scratch.get_params()
print(f"\n📊 Paramètres appris:")
print(f"   w1 = {params['w'][0, 0]:.4f}")
print(f"   w2 = {params['w'][1, 0]:.4f}")
print(f"   b = {params['b']:.4f}")

### Visualisation de la courbe d'apprentissage

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(model_scratch.loss_history, linewidth=2)
plt.xlabel('Itération', fontsize=12)
plt.ylabel('Loss (Binary Cross-Entropy)', fontsize=12)
plt.title('Courbe d\'apprentissage - Classification', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 Observation: La loss diminue et converge.")
print("   Le modèle apprend à séparer les deux classes!")

### Visualisation de la frontière de décision

La frontière de décision est l'ensemble des points où $\hat{y} = 0.5$, c'est-à-dire où $w_1 x_1 + w_2 x_2 + b = 0$.

In [ ]:
def plot_decision_boundary(model, X, y, title="Frontière de décision"):
    """
    Visualiser la frontière de décision.
    """
    # Créer une grille de points
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 100),
                           np.linspace(x2_min, x2_max, 100))
    
    # Prédire sur tous les points de la grille
    X_grid = np.c_[xx1.ravel(), xx2.ravel()]
    Z = model.predict(X_grid)
    Z = Z.reshape(xx1.shape)
    
    # Visualisation
    plt.figure(figsize=(10, 6))
    plt.contourf(xx1, xx2, Z, alpha=0.3, levels=1, colors=['blue', 'red'])
    plt.contour(xx1, xx2, Z, levels=[0.5], colors='black', linewidths=2)
    
    # Points de données
    plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Classe 0', 
                alpha=0.7, edgecolors='k', s=50)
    plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Classe 1', 
                alpha=0.7, edgecolors='k', s=50)
    
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.title(title, fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.show()

# Visualiser
plot_decision_boundary(model_scratch, X, y, 
                      title="Frontière de décision (from scratch)")

print("\n💡 Observation: La ligne noire sépare les deux régions.")
print("   Les régions bleue et rouge correspondent aux prédictions.")

### Évaluation du modèle

In [ ]:
# Prédictions
y_pred = model_scratch.predict(X).flatten()

# Accuracy
accuracy = np.mean(y_pred == y)
print(f"📈 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Matrice de confusion
cm = confusion_matrix(y, y_pred)
print(f"\n📊 Matrice de confusion:")
print(cm)

# Visualisation de la matrice de confusion
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Classe 0', 'Classe 1'],
            yticklabels=['Classe 0', 'Classe 1'],
            cbar_kws={'label': 'Nombre'})
plt.xlabel('Prédiction', fontsize=12)
plt.ylabel('Vraie classe', fontsize=12)
plt.title('Matrice de confusion', fontsize=14)
plt.show()

print(f"\n💡 Interprétation:")
print(f"   • Vrais Négatifs (TN): {cm[0,0]}")
print(f"   • Faux Positifs (FP): {cm[0,1]}")
print(f"   • Faux Négatifs (FN): {cm[1,0]}")
print(f"   • Vrais Positifs (TP): {cm[1,1]}")

## 5. Utilisation de scikit-learn

In [ ]:
# Créer et entraîner le modèle sklearn
model_sklearn = LogisticRegression(random_state=42)
model_sklearn.fit(X, y)

# Paramètres
print(f"📊 Paramètres scikit-learn:")
print(f"   w1 = {model_sklearn.coef_[0, 0]:.4f}")
print(f"   w2 = {model_sklearn.coef_[0, 1]:.4f}")
print(f"   b = {model_sklearn.intercept_[0]:.4f}")

# Prédictions
y_pred_sklearn = model_sklearn.predict(X)
accuracy_sklearn = accuracy_score(y, y_pred_sklearn)

print(f"\n📈 Accuracy (scikit-learn): {accuracy_sklearn:.4f} ({accuracy_sklearn*100:.2f}%)")

# Visualiser la frontière de décision
plot_decision_boundary(model_sklearn, X, y, 
                      title="Frontière de décision (scikit-learn)")

### Rapport de classification détaillé

In [ ]:
print("📊 Rapport de classification (scikit-learn):\n")
print(classification_report(y, y_pred_sklearn, 
                          target_names=['Classe 0', 'Classe 1']))

print("\n💡 Explication des métriques:")
print("   • Precision: Proportion de prédictions positives correctes")
print("   • Recall: Proportion de vrais positifs correctement identifiés")
print("   • F1-score: Moyenne harmonique de precision et recall")
print("   • Support: Nombre d'échantillons dans chaque classe")

## 6. Classification multi-classe

### Exemple : Classification de directions de mouvement (neurosciences)

Simulons un problème de classification de 4 directions de mouvement à partir de l'activité neuronale.

In [ ]:
# Simuler des données multi-classes
np.random.seed(42)

n_trials = 400  # Nombre d'essais
n_neurons = 20  # Nombre de neurones
n_classes = 4   # 4 directions: droite, gauche, haut, bas

# Générer les données
X_multi = np.random.randn(n_trials, n_neurons) * 10 + 30  # Firing rates
y_multi = np.random.randint(0, n_classes, n_trials)

# Ajouter des patterns pour chaque classe
for i in range(n_classes):
    mask = y_multi == i
    # Chaque classe a un pattern distinct
    X_multi[mask, i*5:(i+1)*5] += np.random.randn(np.sum(mask), 5) * 5 + 20

print(f"📊 Dataset multi-classe:")
print(f"   Nombre d'essais: {n_trials}")
print(f"   Nombre de neurones: {n_neurons}")
print(f"   Nombre de classes: {n_classes}")
print(f"\n   Distribution des classes:")
for i in range(n_classes):
    count = np.sum(y_multi == i)
    print(f"      Direction {i}: {count} échantillons")

### Division en ensembles d'entraînement et de test

In [ ]:
# Diviser les données
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(
    X_multi, y_multi, test_size=0.2, random_state=42, stratify=y_multi
)

print(f"📊 Division:")
print(f"   Entraînement: {X_train_multi.shape[0]} échantillons")
print(f"   Test: {X_test_multi.shape[0]} échantillons")

### Entraînement du modèle multi-classe

In [ ]:
# Entraîner un modèle de régression logistique multi-classe
model_multi = LogisticRegression(multi_class='multinomial', max_iter=1000, random_state=42)
model_multi.fit(X_train_multi, y_train_multi)

# Prédictions
y_train_pred_multi = model_multi.predict(X_train_multi)
y_test_pred_multi = model_multi.predict(X_test_multi)

# Accuracy
train_accuracy_multi = accuracy_score(y_train_multi, y_train_pred_multi)
test_accuracy_multi = accuracy_score(y_test_multi, y_test_pred_multi)

print(f"📈 Performance:")
print(f"   Accuracy (entraînement): {train_accuracy_multi:.4f} ({train_accuracy_multi*100:.2f}%)")
print(f"   Accuracy (test): {test_accuracy_multi:.4f} ({test_accuracy_multi*100:.2f}%)")

### Matrice de confusion pour multi-classe

In [ ]:
# Matrice de confusion
cm_multi = confusion_matrix(y_test_multi, y_test_pred_multi)

# Visualisation
plt.figure(figsize=(10, 8))
direction_labels = ['Droite', 'Gauche', 'Haut', 'Bas']
sns.heatmap(cm_multi, annot=True, fmt='d', cmap='Blues',
            xticklabels=direction_labels,
            yticklabels=direction_labels,
            cbar_kws={'label': 'Nombre'})
plt.xlabel('Prédiction', fontsize=12)
plt.ylabel('Vraie direction', fontsize=12)
plt.title('Matrice de confusion - Classification de directions', fontsize=14)
plt.show()

print("\n💡 Interprétation:")
print("   • La diagonale montre les prédictions correctes")
print("   • Les éléments hors-diagonale sont les erreurs")
print("   • On peut identifier quelles directions sont confondues")

### Rapport de classification détaillé

In [ ]:
print("📊 Rapport de classification détaillé:\n")
print(classification_report(y_test_multi, y_test_pred_multi,
                          target_names=direction_labels))

### Analyse des poids : Quels neurones sont importants ?

In [ ]:
# Récupérer les poids
weights_multi = model_multi.coef_  # Shape: (n_classes, n_features)

# Visualisation des poids pour chaque classe
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i in range(n_classes):
    axes[i].bar(range(n_neurons), weights_multi[i], alpha=0.7)
    axes[i].set_xlabel('Neurone', fontsize=11)
    axes[i].set_ylabel('Poids', fontsize=11)
    axes[i].set_title(f'{direction_labels[i]}', fontsize=12)
    axes[i].grid(True, alpha=0.3, axis='y')
    axes[i].axhline(y=0, color='red', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()

print("\n💡 Interprétation des poids:")
print("   • Poids positifs: augmentation de l'activité favorise cette direction")
print("   • Poids négatifs: augmentation de l'activité défavorise cette direction")
print("   • Poids proches de 0: neurone peu informatif pour cette direction")

### Identification des neurones les plus informatifs

In [ ]:
# Calculer l'importance globale de chaque neurone
# (moyenne de la valeur absolue des poids à travers toutes les classes)
neuron_importance = np.mean(np.abs(weights_multi), axis=0)
sorted_neurons = np.argsort(neuron_importance)[::-1]

# Visualisation
plt.figure(figsize=(12, 6))
plt.barh(range(n_neurons), neuron_importance[sorted_neurons], alpha=0.7)
plt.yticks(range(n_neurons), [f'Neurone {i+1}' for i in sorted_neurons])
plt.xlabel('Importance moyenne (|poids|)', fontsize=12)
plt.title('Importance des neurones pour la classification', fontsize=14)
plt.grid(True, alpha=0.3, axis='x')
plt.show()

print(f"\n🧠 Top 5 neurones les plus informatifs:")
for rank, idx in enumerate(sorted_neurons[:5]):
    print(f"   {rank+1}. Neurone {idx+1}: importance = {neuron_importance[idx]:.3f}")

## 7. Exercices pratiques

### À vous de jouer !

**Exercice 1 :** Modifier le seuil de décision
- Au lieu d'utiliser 0.5 comme seuil, essayez 0.3 ou 0.7
- Comment cela affecte-t-il l'accuracy et la matrice de confusion ?

**Exercice 2 :** Augmenter le nombre de directions
- Modifier n_classes à 8 (8 directions)
- Réentraîner le modèle
- Comment la performance change-t-elle ?

**Exercice 3 :** Feature importance
- Identifier les 5 neurones les plus importants
- Entraîner un nouveau modèle avec seulement ces neurones
- Comparer la performance

**Exercice 4 :** Données non linéairement séparables
- Créer un dataset circulaire (make_circles de sklearn)
- Essayer la régression logistique linéaire
- Observer les limites du modèle linéaire

In [ ]:
# Votre code ici
# ...


## 8. Conclusion

Dans ce notebook, nous avons :

✅ Compris la fonction sigmoïde et son rôle dans la classification

✅ Implémenté la régression logistique from scratch

✅ Visualisé les frontières de décision

✅ Utilisé scikit-learn pour une implémentation robuste

✅ Étendu à la classification multi-classe

✅ Appliqué à un problème de neurosciences

✅ Appris à interpréter les matrices de confusion

✅ Analysé l'importance des features (neurones)

### Points clés à retenir:

1. **La régression logistique** utilise la sigmoïde pour transformer les sorties en probabilités
2. **Binary cross-entropy** est la fonction de coût appropriée pour la classification
3. **La frontière de décision** est une ligne (ou hyperplan) qui sépare les classes
4. **Matrice de confusion** permet de voir les types d'erreurs du modèle
5. **Multi-classe** utilise softmax et categorical cross-entropy
6. **Les poids** indiquent l'importance des features pour chaque classe

### Prochaine étape:
Passez au notebook **4.3_exercices.ipynb** pour plus de pratique!